# Chapter 7 데이터 시각화

## 1. 데이터 시각화 이해

### **A. 데이터 시각화**

* 데이터나 정보를 그래프, 차트 등 시각적 형식으로 표현하는 것
* 시각적 이미지는 보다 직관적인 인식을 가능하게 하고 강렬한 인상을 남김
* 다양한 시각화를 통해 데이터에 대한 다층적인 이해 가능
* 빅데이터를 효과적으로 요약하고 전달할 수 있음

### **B. 데이터 유형에 따른 시각화 방법**

|**유형**|**설명**|**예제**|**시각화 방법**|
|:-:|:-|:-:|:-|
|범주형 데이터|- 집단으로 구분되는 데이터 <br>- 기본적으로 숫자로 표현할 수 없음 <br>- 대소비교, 산술연산 불가능|성별, 종교|- 막대 그래프(bar chart) <br>- 원 그래프(pie chart)|
|수치형 데이터|- 수치 값을 가지는 데이터 <br>- 대소비교, 산술연산 가능|키, 몸무게|- 히스토그램(histogram) <br>- 상자 그림(box plot) <br>- 산점도(scatter plot) <br>- 선 그래프(line plot)|

## 2. R base graphics

> 진짜 초보 단계에서 가장 심플하게 할 때 사용. 안씀
>
> 매개변수를 통해 세부적인 그래픽 요소 설정 가능

## 3. ggplot2 패키지

### **A. ggplot2 패키지 소개**

* 범용성, 명료함, 일관성 있는 인터페이스, 미적인 그래픽 결과를 제공하는 패키지
* **독립적인 시각적 구성요소**를 정의하고, **이를 층층히 쌓아올리는 방식**으로 그래프를 생성

1. Data : 데이터 준비
2. Aesthetics : 축에 대한 설정(x축, y축, 크기, 색상, 모양 등)
3. Geometries : 시각화 방법(기하 객체 : 점, 선, 막대 등)
---
4. Facets : 그래프 화면 분할
5. Statistics : 통계량과 함께 표시
6. Coordinates : 그래프의 변형, 회전
7. Theme : 테마

![](https://stat6503.github.io/RP2025/img/fig7-1.png)

### **B. ggplot2 기본 구조**

* 가장 기본적인 구조는 `ggplot()` 함수와 `geom_` 계열 함수로 구성됨
* 각 구성요소는 `+`로 연결되며, 줄바꿈 시에 반드시 `+`는 줄 마지막에 위치해야 함
  * `ggplot()` 함수
    * 좌표 시스템(빈 그래프)을 생성하며, 이후 레이어를 추가하여 그래프를 완성
    * 첫 번째 매개변수는 데이터프레임으로 지정
  * `geom_` 계열 함수
    * 범주형 데이터 시각화 - `geom_bar()`
    * 수치형 데이터 시각화 - `geom_histogram()`, `geom_boxplot()`, `geom_point()`, `geom_line()`

### **C. 막대 그래프(bar chart)**

* 범주형 데이터를 시각화한 것으로, 집단별 빈도를 막대로 나타낸 그래프
* [예제] 다이아몬드 가격 데이터셋 diamonts
    * cut : 컷 품질(Fair, Good, Very Good, Premium, Ideal)
    * color : 다이아몬드 색상(from D(best) to J(worst))
    * clarity : 다이아몬드 투명도(I1(worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF(best))
    > 전부 순서가 있는 범주형 변수